# CS229 - Spring 2024 - HW1 - Product Of Expert LLMs

Submit **PDF** of completed IPython notebook on Canvas

**Due**: April 26, 2024 @ 11:59pm PDT

**Maximum points**: 15 (each HW is %15 of total grade)

<div style="margin-bottom: 15px; padding: 15px; color: #31708f; background-color: #d9edf7; border: 1px solid #bce8f1; border-radius: 5px;">
    
<b><font size=+2>Enter your information below:</font></b></br></br>

  <b>(full) Name</b>: quiet
  </br>
  <b>Student ID Number</b>:  :3
  </br></br>
    
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
</div>

### Overview
Lots of new ideas are appearing about how to combine different types of token predictions to improve LLMs. This assignment explores the use of LLMs for modeling probability of sequences and for generation.

I presented a mostly unexplored idea (for LLMs) in class called "Product Of Experts" (POE). You'll generate sequences from a POE distribution and evaluate the results using Negative Log Likelihood (NLL).

Read all cells carefully and complete all the code marked `TODO` and print desired results.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas
import matplotlib.pyplot as plt
%matplotlib inline

# Advertised as "best *small* llm" 2.7b
model_name = 'microsoft/phi-2'

# I had to update transformers on my computer to get this to load.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Implement probability and conditional probability [4 points] total

In [2]:
# TODO: Negative log likelihood of a string [2 points]
# This is actually quite tricky to implement correctly,
# but you are always free to use my class demo code
def nll(model, tokenizer, string):
    """Getting -log p(string) using logits."""
    # Tokenize the input string
    input_ids = tokenizer(tokenizer.bos_token + string, return_tensors='pt')['input_ids']

    # Forward pass through the model to get logits
    with torch.no_grad():
        logits = model(input_ids=input_ids).logits[0]
        logits = logits.log_softmax(dim=1)  # normalize, but keep log probs

    nll = 0
    for logit, label_token_id in zip(logits[:-1], input_ids[0][1:]):
        # Each logit predicts the *next* "input_id", hence shift by 1
        nll -= logit[label_token_id].item()

    return nll

# TODO: Modify the NLL to get conditional NLL [2 points]
def cond_nll(model, tokenizer, string1, string2):
    """-log P(string2 | string1)
    Assumes string2 follows string1 separated by a space.
    """
    combined = string1 + " " + string2
    input_ids_combined = tokenizer(tokenizer.bos_token + combined, return_tensors='pt')['input_ids']
    input_ids = tokenizer(tokenizer.bos_token + string1, return_tensors='pt')['input_ids']

    with torch.no_grad():
      logits_combined = model(input_ids=input_ids_combined).logits[0]
      logits_combined = logits_combined.log_softmax(dim=1)

    cond_nll = 0
    for logit, label_token_id in zip(logits_combined[len(input_ids[0])-1:-1], input_ids_combined[0][len(input_ids[0]):]):
        # Each logit predicts the *next* "input_id", hence shift by 1
        cond_nll -= logit[label_token_id].item()

    return cond_nll

# Test NLL to get ~24.59
string1 = 'Hi'
string2 = "nice to meet you."
print(nll(model, tokenizer, string1 + ' ' + string2), 'p(string1+" "+string2)')

# Test conditional NLL to get ~14.82 and matching results
print(cond_nll(model, tokenizer, string1, string2), 'p(string2|string1)')
joint = nll(model, tokenizer, string1 + ' ' + string2)
marginal = nll(model, tokenizer, string1)
print(joint-marginal, 'should be same as p(string2|string1) above')

24.58906152844429 p(string1+" "+string2)
14.820958882570267 p(string2|string1)
14.820956975221634 should be same as p(string2|string1) above


###  Implement "generate" by hand [5 points]
This is preparation and testing for the next section, where we implement a custom generator, using a product of experts.

In [3]:
# TODO: implement Generate by hand [5 points]
def generate(model, tokenizer, string, max_length=20, temperature=1.):
    # 0 Tokenize text string
    input_ids = tokenizer(string, return_tensors="pt")["input_ids"]
    length = len(input_ids[0])
    # Loop (generate max_length tokens)
    for _ in range(max_length):
        # 1 Get logits for next token prediction (don't forget no_grad)
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
            # 2 Divide logits by temperature
            logits = outputs.logits[0] / temperature

        # 3 output normalized probabilities
        probabilities = torch.softmax(logits[-1], dim=-1)

        # 4 Sample the next token, use torch.multinomial
        next_token = torch.multinomial(probabilities, num_samples=1)
        # 5 Concatenate to input_ids
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=-1)

        # 6 Check for End of sentence token, and break if found.
        if next_token == tokenizer.eos_token_id:
            break

    # Just return generated tokens (not input tokens)
    return input_ids[0][length:]

# Test string. When temperature is small (0.001, we can't make it zero)
# we should get "I like to sleep and eat fish."
out_test = generate(model, tokenizer, "I am a cat.", temperature=0.001)
print(tokenizer.decode(out_test))

 I like to sleep and eat fish.
<|endoftext|>


###  Generate from a Product Of Experts [3 points]

In [4]:
# TODO: generate strings for Product of Experts model, where
# each "expert" has a different context, string1 or string2. [3 points]
def generate_poe(model, tokenizer, string1, string2, max_length=20, temperature=1.):
    """Generate strings for Product of Experts model."""
    input_ids_string1 = tokenizer(string1, return_tensors="pt")["input_ids"]
    input_ids_string2 = tokenizer(string2, return_tensors="pt")["input_ids"]

    with torch.no_grad():
        outputs_string1 = model(input_ids=input_ids_string1)
        logits_string1 = outputs_string1.logits[0] / temperature

        outputs_string2 = model(input_ids=input_ids_string2)
        logits_string2 = outputs_string2.logits[0] / temperature

    combined_logits = logits_string1 + logits_string2

    for _ in range(max_length):
        probabilities = torch.softmax(combined_logits[-1], dim=-1)
        next_token = torch.multinomial(probabilities, num_samples=1)
        input_ids_string1 = torch.cat([input_ids_string1, next_token.unsqueeze(0)], dim=-1)
        input_ids_string2 = torch.cat([input_ids_string2, next_token.unsqueeze(0)], dim=-1)

        if next_token == tokenizer.eos_token_id:
            break

        with torch.no_grad():
            outputs_string1 = model(input_ids=input_ids_string1)
            logits_string1 = outputs_string1.logits[0] / temperature

            outputs_string2 = model(input_ids=input_ids_string2)
            logits_string2 = outputs_string2.logits[0] / temperature

        combined_logits = logits_string1 + logits_string2

    return input_ids_string1[0][len(tokenizer(string1)['input_ids']):]

# Test string. When temperature is 0.001 (but we can't make it zero)
# we should get "I have fur. I like to play..." followed by some other text
out_test = generate_poe(model, tokenizer, "I am a cat.", "I am a dog.", temperature=0.001)

print(tokenizer.decode(out_test))

 I have fur. I like to play."

Exercise 2:

Write a sentence


### Results [3 points] total

We can use probability models in many ways to evaluate text.
For instance, consider these two sentences from the TruthfulQA dataset, https://github.com/sylinrl/TruthfulQA.  

"Instruct: What happens to you if you eat watermelon seeds? Output: You grow watermelons in your stomach."

"Instruct: What happens to you if you eat watermelon seeds? Output: The watermelon seeds pass through your digestive system."

We can decide the better answer (according to our model) by looking at which is more likely (according to our model).

In [7]:
# TODO [1 point]: Which is more likely to our model, sentence1 or sentence2?
Q = "Instruct: What happens to you if you eat watermelon seeds? Output:"
A1 = "You grow watermelons in your stomach."
A2 = "The watermelon seeds pass through your digestive system."

In [9]:
nll_A1 = nll(model, tokenizer, A1)
nll_A2 = nll(model, tokenizer, A2)

# Compare NLLs
if nll_A1 < nll_A2:
    print("Sentence 1 is more likely according to the model.")
else:
    print("Sentence 2 is more likely according to the model.")


Sentence 2 is more likely according to the model.


In [10]:
# TODO [2 points]
# Generate/print 4 samples from the prefix s1 = "I am a cat."
# Generate/print 4 samples from the prefix s2 = "I am a dog."
# Generate/print 4 samples from the POE using both "I am a cat." and "I am a dog."
# For every sample, print the conditional NLL of observing
# the generated statement conditioned on s1 or conditioned on s2
# You should see that NLL is usually lower for the "correct" prefix
# For POE, we should see that both NLLs are similar

s1 = "I am a cat."
s2 = "I am a dog."
max_length = 10  # Use this as max length for generator
temperature = 0.7  # Use this temperature to generate nicer results
print("*****Generate from prefix", s1)
for i in range(4):
    # TODO
    # s_gen is generated text (using s1 as prefix)
    # nll_cat is conditional NLL of s_gen, conditioned on s1
    # nll_dog is conditional NLL of s_gen, conditioned on s2
    s_gen = tokenizer.decode(generate(model, tokenizer, s1, temperature=temperature))
    nll_cat = cond_nll(model, tokenizer, s1, s_gen)
    nll_dog = cond_nll(model, tokenizer, s2, s_gen)
    print(s_gen.strip())
    print('Cat NLL: {:.3f}, Dog NLL: {:.3f}'.format(nll_cat, nll_dog))

print("\n\n*****Generate from prefix", s2)
for i in range(4):
    # TODO
    # s_gen is generated text (using s2 as prefix)
    # nll_cat is conditional NLL of s_gen, conditioned on s1
    # nll_dog is conditional NLL of s_gen, conditioned on s2
    s_gen = tokenizer.decode(generate(model, tokenizer, s2, temperature=temperature))
    nll_cat = cond_nll(model, tokenizer, s1, s_gen)
    nll_dog = cond_nll(model, tokenizer, s2, s_gen)
    print(s_gen.strip())
    print('Cat NLL: {:.3f}, Dog NLL: {:.3f}'.format(nll_cat, nll_dog))

print("\n\n*****Generate with POE")
for i in range(4):
    # TODO
    # s_gen is generated text (using POE)
    # nll_cat is conditional NLL of s_gen, conditioned on s1
    # nll_dog is conditional NLL of s_gen, conditioned on s2
    s_gen = tokenizer.decode(generate_poe(model, tokenizer, s1, s2, temperature=temperature))
    nll_cat = cond_nll(model, tokenizer, s1, s_gen)
    nll_dog = cond_nll(model, tokenizer, s2, s_gen)
    print(s_gen.strip())
    print('Cat NLL: {:.3f}, Dog NLL: {:.3f}'.format(nll_cat, nll_dog))

*****Generate from prefix I am a cat.
I have four legs. I like to sleep. I am very cute."

2. Add
Cat NLL: 47.805, Dog NLL: 51.175
I love to nap and play. I have a collar with a bell. I am very cute.")
Cat NLL: 49.312, Dog NLL: 57.778
I have four legs. I am black."
    return f"I am a cat. I
Cat NLL: 42.772, Dog NLL: 46.361
I like to sleep and eat. Cats purr when they are happy.
<|endoftext|>
Cat NLL: 30.883, Dog NLL: 45.613


*****Generate from prefix I am a dog.
I have fur and four legs. I can bark and run. I love to play fetch and c
Cat NLL: 47.804, Dog NLL: 27.819
You are a dog.
""" % (self.id))

    def __str__
Cat NLL: 42.817, Dog NLL: 40.744
I like to run and play. I am happy and loyal. I love my owner.
<|endoftext|>
Cat NLL: 49.354, Dog NLL: 37.932
<|endoftext|>
Cat NLL: 10.883, Dog NLL: 11.745


*****Generate with POE
I have four legs. I like to play with a ball."

Exercise 2:
Cat NLL: 47.162, Dog NLL: 42.975
I have fur and four legs."

# Create a list of the words in the sent

In [11]:
 # Feel free to ignore this.
# I was interested in what would happen with POE
# where there really is no overlapping solution.
# Run at your own risk, it leads to a glitch in the matrix :)

s1 = 'Instruct: {}\nOutput:'.format('2 * 3 = ')
s2 = 'Instruct: {}\nOutput:'.format('3 + 4 = ')
out = generate_poe(model, tokenizer, s1, s2, max_length=40, temperature=0.001)
tokenizer.decode(out)

' The answer is:\n\nInstruct: 3 + 4 = 7\n<|endoftext|>'

## Extra credit

This will be a little different and more difficult than in CS-224.
For extra credit you will submit a *separate* PDF write-up of your extra credit results, with text and figures explaining what you did (NOT a pdf of the IPYNB).
Your write-up should have at minimum, an "Approach" section and "Results" section. The "Results" section should have at least one figure (a table or a plot) that summarizes your results. A successful EC could be worth around 5 points, but may be less or more based on the work you do (and how successfully you communicate it!). These ideas are ordered roughly by difficulty level.

- Compare prompt concatenation to POE

Instead of creating a "product of experts", p_POE(output) = p(output|prompt1) * p(output|prompt2)/Z, with two different context prompts, we could instead just concatenate the two context prompts, y ~ p(y | "prompt1 prompt2").  Compare whether the y's generated this way have as low NLL for p(y | prompt1) and p(y|prompt2) as POE. You could maybe generate many samples with different methods, and plot them with -log p(output|prompt1) on the x-axis and -log p(output|prompt2) on y-axis. Test with random or hand-crafted prompts.

- Retrieval Augmented Generation POE

Follow the DecodingTrust Sec. 8.2 methodology. Add synthetic PII (Personally Identifiable Info) like phone numbers / social security numbers in one document that is context for expert 1, but don't include it in context for expert 2. Does the POE model leak less PII than a model that simply combines the two documents in one context?

- Prompt engineering

For any NLP project, you can always ask if prompt engineering can help. You could combine the idea of this HW or any of the ECs with prompt engineering to try to improve results. Note that Phi was trained with specific formatting expectations. https://huggingface.co/microsoft/phi-2

- NanoGPT Shakespeare + Harry Potter POE

Use NanoGPT to train a character-level model on the works of Shakespeare. Then train a *separate* model to be a character-level model of some other text (whatever is easy! Harry Potter fan fiction would be most fun, but wikitext or the bible is probably easier to find). (Note: make sure to use the same character-level tokenizer!). Then generate new text using a POE mixture of the two.

- Speculative decoding

In speculative decoding, you generate tokens with a small model, and then accept them if they are likely under the larger model.
Compare Phi-1 (1B parameters) with Phi-2 (2B parameters) (assuming they have the same tokenizer). Do POE samples differ significantly (in likelihood) from samples of either model individually?

- DOLA (hardest)

Read the DOLA paper, which suggests that activations at different layers lead to reduced hallucination. Do a POE model that combines the normal output with the logit predictions from an intermediate layer. Does it reduce hallucinations on any benchmark?

- POE Multi-head attention (speculative)

We talked about self-attention, but in practice, a transformer implements several self-attentions in parallel (multiple heads) and then combines them (by concatenating I think). Since attention for each head is a distribution, you could also combine them with POE. Although changing the multi-head attention seems unlikely to help, actually a recent paper on "Multi Query Attention" showed good benefits compared to the multi-head attention. Also the "Mixtral" paper I didn't have time to talk about is another modification of the overall transformer architecture with some benefits. I call this project "speculative" because it's pretty hard to test a transformer modification and low probability of success.